# Analysis of H.E.S.S. DL3 data with Gammapy

In September 2018 the [H.E.S.S.](https://www.mpi-hd.mpg.de/hfm/HESS) collaboration released a small subset of archival data in FITS format. This tutorial explains how to analyse this data with Gammapy.

We will analyse four observation runs of the Crab nebula, which are part of the [H.E.S.S. first public test data release](https://www.mpi-hd.mpg.de/hfm/HESS/pages/dl3-dr1/). The data was release without correspoding background models. In [background_model.ipynb](background_model.ipynb) we show how to make a simple background model, which is also used in this tutorial. The background model is not perfect; it assumes radial symmetry and is in general derived from only a few observations.

**Note:** The high level `Analysis` class is a new feature added in Gammapy v0.14. In the curret state it supports the standard analysis cases of a joint or stacked 3D and 1D analysis. It provides only limited access to analaysi parameters via the config file. It is expected that the format of the YAML config will be extended and change in future Gammapy versions.

We will first show how to configure and run a stacked 3D analysis and then address the classical spectral analysis using reflected regions later. The structure of the tutorial follows a typical analysis

- Analysis configuration
- Observation slection
- Data reduction
- Model fitting

## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import yaml
from pathlib import Path
from regions import CircleSkyRegion
from astropy import units as u
from astropy.coordinates import SkyCoord
from gammapy.scripts import Analysis

## Analysis configuration

For configuration of the analysis we use the [YAML](https://en.wikipedia.org/wiki/YAML) data format. YAML is a machine readable serialisation format, that is also friendly for humans to read. In this tutorial we will write the configuration file just using Python strings, but of course it can be created and modified with any text editor of your choice.

We start with the section of `"general"` settings:

In [ ]:
config = """
general:
    logging:
        level: INFO
    outdir: .

observations:
    datastore: $GAMMAPY_DATA/hess-dl3-dr1/hess-dl3-dr3-with-background.fits.gz
    filters:
        - filter_type: par_value
          value_param: Crab
          variable: TARGET_NAME

reduction:
    dataset-type: MapDataset
    stack-datasets: true
    offset-max: 2.5 deg
    psf-kernel-radius: 0.3 deg
    geom:
        skydir: [83.633, 22.014]
        width: [5, 5]
        binsz: 0.02
        coordsys: CEL
        proj: TAN
        axes:
          - name: energy
            hi_bnd: 30
            lo_bnd: 1
            nbin: 5
            interp: log
            node_type: edges
            unit: TeV
    geom-irf:
        skydir: [83.633, 22.014]
        width: [5, 5]
        binsz: 0.2
        coordsys: CEL
        proj: TAN
        axes:
            - name: energy
              hi_bnd: 30
              lo_bnd: 1
              nbin: 11
              interp: log
              node_type: center
              unit: TeV

model: model.yaml

fit:
    fit_range:
        max: 100 TeV
        min: 1 TeV

flux: {}
"""
filename = Path("config.yaml")
filename.write_text(config);

In [ ]:
model_config = """
components:
- name: crab
  type: SkyModel
  spatial:
    type: PointSpatialModel
    frame: icrs
    parameters:
    - name: lon_0
      value: 83.63
      unit: deg
    - name: lat_0 
      value: 22.14    
      unit: deg
  spectral:
    type: PowerLawSpectralModel
    parameters:
    - name: amplitude      
      value: 1.0e-12
      unit: cm-2 s-1 TeV-1
    - name: index
      value: 2.0
      unit: ''
    - name: reference
      value: 1.0
      unit: TeV
      frozen: true
"""
filename = Path("model.yaml")
filename.write_text(model_config);

##  Observation selection

We first create the high level `Analysis` object from the yaml config file, using the `.from_yaml()` method:

In [ ]:
analysis = Analysis.from_yaml("config.yaml")

In [ ]:
analysis.get_observations()

In [ ]:
analysis.observations.ids

In [ ]:
analysis.observations["23592"]

## Data reduction

In [ ]:
%%time
analysis.get_datasets()

In [ ]:
analysis.datasets.names

In [ ]:
print(analysis.datasets["stacked"])

In [ ]:
counts = analysis.datasets["stacked"].counts

In [ ]:
print(counts)

In [ ]:
counts.smooth("0.05 deg").plot_interactive()

## Model fitting

In [ ]:
analysis.get_model()

In [ ]:
print(analysis.model)

In [ ]:
print(analysis.model["crab"])

In [ ]:
analysis.run_fit()

In [ ]:
print(analysis.fit_result)

In [ ]:
analysis.model.to_yaml("model-best-fit.yaml")

In [ ]:
!cat model-best-fit.yaml

### Inspecting residuals

In [ ]:
analysis.datasets["stacked"].plot_residuals(method="diff/sqrt(model)", vmin=-0.5, vmax=0.5);

In [ ]:
region = CircleSkyRegion(center=SkyCoord("83.63 deg", "22.14 deg"), radius=0.5 * u.deg)

In [ ]:
analysis.datasets["stacked"].plot_residuals(region=region, method="diff/sqrt(model)", vmin=-0.5, vmax=0.5);

In [ ]:
residuals = analysis.datasets["stacked"].residuals(method="diff")
residuals.smooth("0.08 deg").plot_interactive(cmap="coolwarm", vmin=-0.1, vmax=0.1, stretch="linear", add_cbar=True)

### Inspecting likelihood profiles

In [ ]:
profile = analysis.fit.likelihood_profile(parameter="lon_0")

In [ ]:
total_stat = analysis.fit_result.total_stat
plt.plot(profile["values"], profile["likelihood"] - total_stat)
plt.xlabel("Lon (deg)")
plt.ylabel("Delta TS")

## Exercises

- Run a spectral analysis using reflected regions. You can use `Analysis.from_template("1d")` to get an example configuration file. Add the resulting flux points to the SED plotted above.
